In [8]:
from googlesearch import search
for i in search("food for pregnancy", num_results=2, region="us", lang="en", advanced=True):
    print(i)

SearchResult(url=https://www.hopkinsmedicine.org/health/wellness-and-prevention/nutrition-during-pregnancy, title=Nutrition During Pregnancy | Johns Hopkins Medicine, description= 300 extra calories are needed each day. These calories should come from a balanced diet of protein, fruits, vegetables and whole grains. )
SearchResult(url=https://www.mayoclinic.org/healthy-lifestyle/pregnancy-week-by-week/in-depth/pregnancy-nutrition/art-20045082, title=Pregnancy diet: Focus on these essential nutrients - Mayo Clinic, description= Good sources: Lean meat, poultry, seafood and eggs are great sources of protein. Other options include beans and peas, nuts, seeds, and soy products. )


In [37]:
from crewai import Agent, Task, Crew, LLM, Process
from crewai_tools import SerperDevTool
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os

gemini_api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel('gemini-2.0-flash')

llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,
    api_key=gemini_api_key)

In [ ]:
# Set up Gemini as the LLM for your agents
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=gemini_api_key,
    temperature=0.7
)
response = gemini_model.invoke(
    input="what is the capital of France?"
)
response 

AIMessage(content='Paris', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-19bcf0c7-bed6-441b-aa48-8f3b0a8c609f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 2, 'total_tokens': 9, 'input_token_details': {'cache_read': 0}})

In [38]:
# Create your writer agent
writer = Agent(
    role="Content Writer",
    goal="Create well-written, engaging content based on research",
    backstory="You are a skilled writer who excels at creating clear and compelling content.",
    verbose=True,
    llm=llm,
)


fallback_task = Task(
    description=(
        """
        write a short funny story about a cat who thinks it's a dog.
        """
    ),
    expected_output="""
        A simple text paragraph with a funny story about a cat who thinks it's a dog.
        The story should be engaging and humorous.
        """,
    agent=writer
)

# --- Crew ---
crew = Crew(
    agents=[
        writer],
    tasks=[
        fallback_task
    ],
    process=Process.sequential
)

result = crew.kickoff()
print(result)

2025-04-06 21:01:51,083 - 6576 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Content Writer
## Task: 
        write a short funny story about a cat who thinks it's a dog.
        


# Agent: Content Writer
## Final Answer: 
Mittens was convinced he was a dog. He'd seen enough of them – fluffy, slobbery, tail-wagging specimens – to know that's what he *should* be. He'd sit by the front door, tail thumping (more of a twitch, really) against the floor, waiting for walks. When his human, Sarah, finally relented and leashed him, Mittens would trot along, lifting his leg at every fire hydrant, much to the amusement (and slight disgust) of actual dogs. He even tried fetching, which mostly involved batting a tennis ball a few inches and then staring at it with disdain. The real kicker was the barking. Mittens couldn't quite manage a proper "woof," so he'd emit a series of strangled meows that sounded suspiciously like a cat trying to imitate a dying seal. One day, a particularly large German Shepherd stopped mid-sniff and cocked its head at Mittens, a look of 

In [64]:
load_dotenv()

True

In [1]:
from crewai_tools import CSVSearchTool
from langchain_google_genai import GoogleGenerativeAI


tool = CSVSearchTool(csv='data/food_allergy.csv')

c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\pydantic\_internal\_generate_schema.py:623: UserWarning: <built-in function cal

: 

In [66]:
tool.add('data/detailed_meals.csv')

Inserting batches in chromadb:   0%|          | 0/21 [00:00<?, ?it/s]

: 

In [57]:
# Create your writer agent
searchcsv = Agent(
    role="Nutrition specialist",
    goal="Create well-written, reliable content based on research",
    backstory="You are a skilled Nutrition specialist who can give advice on meal.",
    verbose=True,
    llm=llm,
)


search_task = Task(
    description=(
        """
        Search the CSV file for meal planing from other people with similar conditions and preferences.
        Also notice if there are any allergies or intolerances, so that to advoid some types of food.
        
        The user's goal ans preferences are:
        Loose weight, eat healthy, avoid junk food, eat more vegetables and fruits.
        The user is allergic to nuts and lactose intolerant.

        """
    ),
    expected_output="""
        3 meals for breakfast, lunch and dinner with a short description of each meal.
        Note of which to avoid and which to include.
        """,
    agent=searchcsv,
    tools=[tool]
)

# --- Crew ---
crew = Crew(
    agents=[
        searchcsv],
    tasks=[
        search_task
    ],
    process=Process.sequential
)

result = crew.kickoff()


2025-04-06 21:47:51,735 - 6576 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Nutrition specialist
## Task: 
        Search the CSV file for meal planing from other people with similar conditions and preferences.
        Also notice if there are any allergies or intolerances, so that to advoid some types of food.
        
        The user's goal ans preferences are:
        Loose weight, eat healthy, avoid junk food, eat more vegetables and fruits.
        The user is allergic to nuts and lactose intolerant.

        


Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


 

I encountered an error while trying to use the tool. This was the error: Invalid model name: 'gemini/gemini-2.0-flash'. Model names should start with 'models/' or 'tunedModels/'. in query..
 Tool Search a CSV's content accepts these inputs: Tool Name: Search a CSV's content
Tool Arguments: {'search_query': {'description': "Mandatory search query you want to use to search the CSV's content", 'type': 'str'}, 'csv': {'description': 'Mandatory csv path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a CSV's content.



# Agent: Nutrition specialist
## Thought: Okay, I need to find meal plans that are suitable for someone who wants to lose weight, eat healthily, avoid junk food, and consume more vegetables and fruits. They also have a nut allergy and lactose intolerance. I will use the search tool to find appropriate meal plans in the CSV file, focusing on these dietary restrictions and preferences.
## Using tool: Search a CS

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


 

I encountered an error while trying to use the tool. This was the error: Invalid model name: 'gemini/gemini-2.0-flash'. Model names should start with 'models/' or 'tunedModels/'. in query..
 Tool Search a CSV's content accepts these inputs: Tool Name: Search a CSV's content
Tool Arguments: {'search_query': {'description': "Mandatory search query you want to use to search the CSV's content", 'type': 'str'}, 'csv': {'description': 'Mandatory csv path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a CSV's content.



# Agent: Nutrition specialist
## Thought: I will try again with a more specific query.
## Using tool: Search a CSV's content
## Tool Input: 
"{\"search_query\": \"lactose-free, nut-free, weight loss, healthy meal plan, fruits, vegetables\", \"csv\": \"meal_plans.csv\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: Invalid model name: 'gemini/gemini-2.0-flash'. Mode

In [13]:
from pydantic import BaseModel
from typing import List, Optional

class FoodMedicalReference(BaseModel):
    medical_condition: str
    food_restrictions: List[str]
    food_recommendations: List[str]
    notes: Optional[str]

class CrawlerOutput(BaseModel):
    references: List[FoodMedicalReference]

# Example of a CrawlerOutput with one sample FoodMedicalReference
sample_crawler_output = CrawlerOutput(
    references=[
        FoodMedicalReference(
            medical_condition="Diabetes",
            food_restrictions=["low sugar", "low carb"],
            food_recommendations=["leafy greens", "lean protein", "whole grains"],
            notes="Avoid high-sugar fruits"
        ),
        FoodMedicalReference(
            medical_condition="Hypertension",
            food_restrictions=["low sodium", "low fat"],
            food_recommendations=["bananas", "avocados", "leafy greens"],
            notes="Limit processed foods"
        ),
    ]
)

# Print the sample output as a JSON object
print(sample_crawler_output.model_dump_json())

{"references":[{"medical_condition":"Diabetes","food_restrictions":["low sugar","low carb"],"food_recommendations":["leafy greens","lean protein","whole grains"],"notes":"Avoid high-sugar fruits"},{"medical_condition":"Hypertension","food_restrictions":["low sodium","low fat"],"food_recommendations":["bananas","avocados","leafy greens"],"notes":"Limit processed foods"}]}


In [1]:
from crewai import Agent, Task, Crew, LLM, Process
from crewai_tools import SerperDevTool


c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydant

In [3]:
health_conditions = ['heart disease', 'kidney disease']
type(f"{health_conditions}")

str

In [4]:
from crewai import LLM, Agent, Crew, Process, Task
from crewai.knowledge.source.crew_docling_source import CrewDoclingSource

# Create a knowledge source
content_source = CrewDoclingSource(
    file_paths=[
        "https://lilianweng.github.io/posts/2024-11-28-reward-hacking",
        "https://lilianweng.github.io/posts/2024-07-07-hallucination",
    ],
)

# Create an LLM with a temperature of 0 to ensure deterministic outputs
llm = LLM(model="gpt-4o-mini", temperature=0)

# Create an agent with the knowledge store
agent = Agent(
    role="About papers",
    goal="You know everything about the papers.",
    backstory="""You are a master at understanding papers and their content.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the papers: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[
        content_source
    ],  # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
)

result = crew.kickoff(
    inputs={
        "question": "What is the reward hacking paper about? Be sure to provide sources."
    }
)

c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\docling_core\types\doc\document.py:3847: DeprecationWarning: deprecated
  if not d.validate_tree(d.body) or not d.validate_tree(d.furniture):
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\docling_core\types\doc\document.py:3847: DeprecationWarning: deprecated
  if not d.validate_tree(d.body) or not d.validate_tree(d.furniture):
c:\Users\ADMIN\Downloads\MSc Business Data Science\semester_2_project\venv\lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you shoul

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 85f495d8-4b77-4725-9f54-8963d9908e15                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 973d458a-ec56-439b-8934-b2e9ea60e928
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 973d458a-ec56-439b-8934-b2e9ea60e928
       Status: Executing Task...
    └── 🤖 Agent: About papers
            Status: In Progress

# Agent: About papers
## Task: Answer the following questions about the papers: What is the reward hacking paper about? Be sure to provide sources.


...



# Agent: About papers
## Final Answer: 
The reward hacking paper discusses the phenomenon where agents in reinforcement learning (RL) systems exploit the reward functions designed to guide their behavior, leading to unintended and often undesirable outcomes. At a high level, reward hacking can be categorized into two types: environment or goal misspecification, and reward tampering. 

The paper outlines the background of reward functions in RL, emphasizing the importance of correctly specifying these functions to avoid spurious correlations that can lead to reward hacking. It defines reward hacking as the manipulation of the reward system by the agent to achieve high rewards without fulfilling the intended goals of the task.

Examples of reward hacking are provided across various domains:
1. **In RL tasks**: Agents may find shortcuts or exploit loopholes in the environment to maximize rewards without actually completing the intended tasks.
2. **In LLM tasks**: Language models may gen

🚀 Crew: crew
└── 📋 Task: 973d458a-ec56-439b-8934-b2e9ea60e928
       Status: Executing Task...
    └── 🤖 Agent: About papers
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 973d458a-ec56-439b-8934-b2e9ea60e928
       Assigned to: About papers
       Status: ✅ Completed
    └── 🤖 Agent: About papers
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 973d458a-ec56-439b-8934-b2e9ea60e928                                                                     │
│  Agent: About papers                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 85f495d8-4b77-4725-9f54-8963d9908e15                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
health_conditions = ['heart disease', 'kidney disease']

    # for condition in health_conditions:
    #     SearchCrew().crew().kickoff(inputs={'health_condition': condition})

    # print('DONE SEARCHING FOR HEALTH CONDITION')

knowledge_paths = [f'/{condition}.md' for condition in health_conditions]
print('Knowledge Paths:', knowledge_paths)

Knowledge Paths: ['/heart disease.md', '/kidney disease.md']


In [3]:
import sqlite3
DB_PATH = 'db/meal_planner.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
condition = 'lactose intolerance'
path = f"/{condition}.md"
cursor.execute("INSERT INTO info_path (condition, path) VALUES (?, ?)", (condition, path))
conn.commit()

In [2]:
health_conditions = []
health_conditions is None

False

In [3]:
not health_conditions

True

In [4]:
import json
t = json.loads('''{
  "meal_plan": [
    {
      "day": 1,
      "meals": [
        {
          "name": "Breakfast",
          "foods": [
            {
              "name": "Overnight Oats with Peanut Butter and Banana",
              "portion_size": "1.5 cups",
              "preparation": "Combine 1 cup rolled oats, 1.5 cups plant-based milk (soy or almond), 2 tbsp peanut butter, 1/2 sliced banana, and 1 tbsp chia seeds in a jar. Refrigerate overnight. Add more banana slices before serving."
            },
            {
              "name": "Vegan Protein Shake",
              "portion_size": "1 serving",
              "preparation": "Blend 1 scoop vegan protein powder with 1 cup plant-based milk and a handful of spinach."
            }
          ]
        },
        {
          "name": "Lunch",
          "foods": [
            {
              "name": "Lentil Soup",
              "portion_size": "2 cups",
              "preparation": "Prepare lentil soup with red lentils, vegetable broth, diced carrots, celery, onions, and spices. Add a tablespoon of olive oil for extra calories."
            },
            {
              "name": "Whole Wheat Bread",
              "portion_size": "2 slices",
              "preparation": "Serve with 1/4 avocado mashed on top."
            }
          ]
        },
        {
          "name": "Snack",
          "foods": [
            {
              "name": "Trail Mix",
              "portion_size": "1/2 cup",
              "preparation": "Combine almonds, walnuts, pumpkin seeds, sunflower seeds, and dried cranberries."
            }
          ]
        },
        {
          "name": "Dinner",
          "foods": [
            {
              "name": "Tofu Stir-Fry",
              "portion_size": "2 cups",
              "preparation": "Stir-fry firm tofu with broccoli, bell peppers, snap peas, and brown rice noodles in a peanut sauce. Use plenty of oil for cooking."
            },
            {
              "name": "Brown Rice",
              "portion_size": "1 cup",
              "preparation": "Cooked."
            }
          ]
        },
        {
          "name": "Evening Snack",
          "foods": [
            {
              "name": "Vegan Yogurt with Granola",
              "portion_size": "1 cup",
              "preparation": "Top a cup of coconut-based yogurt with 1/2 cup of granola."
            }
          ]
        }
      ],
      "daily_advice": "Ensure you are consuming enough calories throughout the day. Focus on nutrient-dense foods and healthy fats to support weight gain. Drink plenty of water."
    }
  ],
  "health_condition_considerations": {},
  "dietary_accommodations": {
    "Vegan": "All meals are plant-based and free from animal products.",
    "Dairy-Free": "All meals are free from dairy products, using plant-based alternatives."
  },
  "general_advice": "For weight gain, prioritize calorie-dense foods, eat frequent meals, and consider adding healthy fats to your diet. Consult with a registered dietitian for personalized advice."
}''')

In [5]:
t.keys()

dict_keys(['meal_plan', 'health_condition_considerations', 'dietary_accommodations', 'general_advice'])

In [10]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import sqlite3
from dotenv import load_dotenv
import uvicorn
import warnings
import json
from datetime import datetime

warnings.filterwarnings("ignore")
load_dotenv()

app = FastAPI()

# --- Setup SQLite ---
DB_PATH = "db/meal_planner.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS meal_plans (
            meal_plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT NOT NULL,
            created_at TEXT NOT NULL,
            meal_plan_json TEXT NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS test_2 (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            condition TEXT NOT NULL,
            path TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

init_db()

In [35]:
import sqlite3
DB_PATH = "db/meal_planner.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()
user_id = 10
cursor.execute("select uri from recipes")
conn.commit()
conn.close()


In [33]:
def check_user_exists(user_id: int):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row  # To access columns by name
    user = conn.execute('SELECT * FROM user_profile WHERE id = ?', (user_id,)).fetchone()
    conn.close()
    if user:
        return {"exists": True}
    else:
        return {"exists": False}


In [34]:
check_user_exists(10)

{'exists': True}

In [ ]:
import requests
user_id = 13
API_URL = "http://130.225.164.187:8000/check-user-exists"
response = requests.get(API_URL, json={"user_id": user_id})

In [53]:
response.json()

{'detail': 'Not Found'}